# ConvGRU for Fluo Classification

This tutorial was very helpful, and some code here is taken from it: https://colab.research.google.com/drive/1B5KQvPySqYEa6XicRHdOwgv8fN1BrCgQ#scrollTo=LI6JtYwTt2HM

In [1]:
# https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab/48919022
import os
from google.colab import files
src = list(files.upload().values())[0]
open('utils.py','wb').write(src)
# print(os.path.abspath('utils.py'))
import utils
# help(utils)

Saving utils.py to utils (4).py


Code to make sure the Drive mount later will work

In [2]:
# utils.prepare_drive()
# https://github.com/googlecolab/colabtools/issues/1494
!sed -i -e 's/enforce_single_parent:true/enforce_single_parent:true,metadata_cache_reset_counter:4/' /usr/local/lib/python3.6/dist-packages/google/colab/drive.py
from google.colab import drive
import importlib
_ = importlib.reload(drive)

This code makes sure the shared drive mount (later on) will work:

In [3]:
# utils.prepare_drive()
# from google.colab import drive

### Install Dependencies

In [4]:
%matplotlib inline

Install Conda

In [5]:
!which python
!python --version
!echo $PYTHONPATH

/usr/local/bin/python
Python 3.6.9
/env/python


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

print("Pytorch version: ", torch.__version__)

Pytorch version:  1.7.0+cu101


In [7]:
import pandas as pd
import numpy as np
import os, time, shutil
import matplotlib.pyplot as plt
from PIL import Image

### Mount Google Drive

In [8]:
import os
from google.colab import drive   

drive.flush_and_unmount()
# mount the google drive to my Colab session
drive.mount('/content/gdrive')
# use the google drive in my Colab session
home_path = '/content/gdrive/Shared drives/Embryo_data'
print(os.listdir(home_path))

Mounted at /content/gdrive
['mxnet_cnn2d_embryo_58_fine_tune_data_aug_ResNet50_v2_order_random.ipynb', 'Embryo3', 'Embryo12', 'Embryo13', 'Embryo16', 'Embryo19', 'Embryo18', 'Embryo24', 'Embryo39', 'Embryo42', 'Embryo46', 'Embryo47', 'Embryo23', 'Embryo33', 'Embryo25', 'Embryo95', 'Embryo97', 'Embryo96', 'Embryo98', 'Embryo101', 'Embryo99', 'Embryo100', 'Embryo102', 'Embryo76', 'Embryo78', 'Embryo81', 'Embryo79', 'Embryo80', 'Embryo84', 'Embryo85', 'Embryo87', 'Embryo88', 'Embryo92', 'Embryo94', 'Embryo93', 'embryo_info_CS101.xlsx', 'data', 'processed', 'models', 'pix2pix_PyTorch-GAN.ipynb', 'annotation.xlsx', 'Embryo110', 'Embryo109', 'Embryo111', 'Embryo113', 'Embryo112', 'Embryo114', 'Embryo116', 'Embryo115', 'Embryo117', 'Embryo118', 'Embryo119', 'Embryo120', 'Embryo103', 'Embryo104', 'Embryo105', 'Embryo107', 'Embryo106', 'Embryo108']


### Get Data

In [9]:
# Fixing the random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Available high quality data
embryo_inds = [1, 3, 12, 13, 16, 18, 19, 24, 39, 40, 42, 45, 46, 47, 49, 50, 52]

# Directory of the processed *.npy files
processed_path = f'{home_path}/processed'
polar_processed_path = f'{processed_path}/polarization'

video_time_info, train_embryos, val_embryos, test_embryos = utils.split_train_test_val(home_path, embryo_inds)

# z_agg_mode = "3 Z Slices Around Middle"
z_agg_mode = "Max Z Timeseries"
data_path = f'{home_path}/data/fluo_data'
pol_path = f'{processed_path}/polarization'

# save_path = f'{processed_path}/fluo_data/3d'
save_path = f'{processed_path}/fluo_data/max'
train_path = os.path.join(save_path, 'train')
val_path = os.path.join(save_path, 'val')
test_path = os.path.join(save_path, 'test')

print(save_path)
print(train_path)
print(val_path)
print(test_path)

              t_num  first_anno_pol_time
embryo_index                            
1                21                   17
3                21                   12
12              143                   11
13              143                   12
16              143                   30
18              143                   27
19              143                   32
24               21                   20
39               21                   12
40               21                    8
42               21                   16
45               21                   19
46               21                   19
47               21                   17
49               21                   21
50               21                   21
52               21                   13
[1, 3, 18, 50, 45, 49, 39, 47, 12, 40, 52, 16, 24, 42]
[46]
[13, 19]
/content/gdrive/Shared drives/Embryo_data/processed/fluo_data/max
/content/gdrive/Shared drives/Embryo_data/processed/fluo_data/max/train
/content/gdriv

### Save NP Data in Correct Format for Training

#### Sanity Check on Embryo npy Shape

In [10]:
embryo_idx = 3
embryo_path = f'{data_path}/embryo{embryo_idx}.npy'
embryo_pol_path = f'{pol_path}/embryo{embryo_idx}.npy'

embryo = np.load(embryo_path)
embryo_pol = np.squeeze(np.load(embryo_pol_path)).astype(int)

# pre-normalization steps
# embryo is currently np.float32, but for 3D input
# this conversion step exceeds Colab RAM usage
embryo = embryo.astype(np.float64)

# print(embryo.shape)

# normalize the data to 0 - 1
normalize = 'per_embryo'
if normalize == 'per_embryo': # max val over the full embryo
    embryo /= np.max(embryo)
elif normalize == 'per_timestep': # max val over each timestep
    embryo /= np.max(embryo, axis=(0,1))
elif type(normalize) is not str: # a fixed numerical factor
    embryo /= normalize
embryo = 255 * embryo # Now scale by 255
embryo = embryo.astype(np.uint8)

print("(z, h, w, t)")
print(embryo.shape)

(z, h, w, t)
(21, 512, 512, 21)


In [11]:
# Actually create the train files
specs = (data_path, pol_path, video_time_info)

# TODO: write function to select z index and use whole timeseries
# for example, if max, then make it into shape (t, c, h, w)
# corresponding to t range (start with all t, could split into before and after polarization though)
# ..., c for channels (z slices), and height h and width w 

# utils.save_nps_subset_z(train_embryos, train_path, specs, window=None, normalize='per_embryo', mode='middle', n_slices=3)
# utils.save_nps_subset_z(test_embryos, test_path, specs, window=None, normalize='per_embryo', mode='middle', n_slices=3)
# utils.save_nps_subset_z(val_embryos, val_path, specs, window=None, normalize='per_embryo', mode='middle', n_slices=3)

### Set Hyperparameters

In [12]:
num_classes = 2

epochs = 15
lr = 0.001
per_device_batch_size = 20
momentum = 0.9
wd = 0.0001

lr_factor = 0.75
lr_steps = [10, 20, 30, np.inf]

num_workers = 1
num_gpus = 1
gpus = [i for i in range(num_gpus)]

device = torch.device(gpus[0]) if num_gpus > 0 else [torch.device('cpu')]

batch_size = per_device_batch_size * max(num_gpus, 1)
print(device)

cuda:0


Things to keep in mind:

1. ``epochs = 5`` is just for this tutorial with the tiny dataset. please change it to a larger number in your experiments, for instance 40.
2. ``per_device_batch_size`` is also set to a small number. In your experiments you can try larger number like 64.
3. remember to tune ``num_gpus`` and ``num_workers`` according to your machine.
4. A pre-trained model is already in a pretty good status. So we can start with a small ``lr``.

### Data Augmentation

In transfer learning, data augmentation can also help.
We use the following augmentation in training:

2. Randomly crop the image and resize it to 224x224
3. Randomly flip the image horizontally
4. Randomly jitter color and add noise
5. Transpose the data from height*width*num_channels to num_channels*height*width, and map values from [0, 255] to [0, 1]
6. Normalize with the mean and standard deviation from the ImageNet dataset.




In [13]:
# TODO: tranform the "video"

transform = transforms.Compose([
    transforms.Resize(600),
    transforms.CenterCrop(512),

    transforms.RandomHorizontalFlip(), # Randomly flip the image horizontally
    transforms.RandomVerticalFlip(),
    # transforms.RandomLighting(0.1), # Add AlexNet-style PCA-based noise to an image... need to implement: https://github.com/facebook/fb.resnet.torch/blob/master/datasets/transforms.lua#L183
    # transforms.functional.adjust_contrast(contrast_factor=0.9 + np.random.random_sample()*0.2),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Data Loaders

With the data augmentation functions, we can define our data loaders:



In [17]:
def check_npy_in_train(filename):
    if not filename.lower().endswith('.npy'):
        return False
    if int("".join(list(filter(lambda x: x.isdigit(), filename)))) in train_embryos:
        return True
    return False

def check_npy_in_val(filename):
    if not filename.lower().endswith('.npy'):
        return False
    if int("".join(list(filter(lambda x: x.isdigit(), filename)))) in val_embryos:
        return True
    return False

def check_npy_in_test(filename):
    if not filename.lower().endswith('.npy'):
        return False
    if int("".join(list(filter(lambda x: x.isdigit(), filename)))) in test_embryos:
        return True
    return False

def check_npy(filename):
    if filename.lower().endswith('.npy'):
        return True
    print(filename)
    return False

def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

def check_png(path):
    return filename.lower().endswith('.png')

def npy_loader(path):
    sample = torch.from_numpy(np.load(path))
    return sample
    
# TODO: Segment Embryo_data/data/fluo_data folders into subfolders for each class... how to define though?
# .... before and after polarization?
train_data = torchvision.datasets.DatasetFolder(root=data_path, loader=npy_loader, is_valid_file=check_npy_in_train)
val_data = torchvision.datasets.DatasetFolder(root=data_path, loader=npy_loader, is_valid_file=check_npy_in_val)
test_data = torchvision.datasets.DatasetFolder(root=data_path, loader=npy_loader, is_valid_file=check_npy_in_test)

# train_data = torchvision.datasets.ImageFolder(root=train_path, transform=transform, target_transform=None, is_valid_file=None)
# val_data = torchvision.datasets.ImageFolder(root=val_path, transform=transform, target_transform=None, is_valid_file=None)
# test_data = torchvision.datasets.ImageFolder(root=test_path, transform=transform, target_transform=None, is_valid_file=None)

print("Train data: ", train_data)
print("Validation data: ", val_data)
print("Test data: ", test_data)

# Define data loaders
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, sampler=None, pin_memory=False, drop_last=False)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, sampler=None, num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, sampler=None, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, sampler=None, num_workers=num_workers, drop_last=True)

classes = ('unpolarized', 'polarized')

RuntimeError: ignored

Visualize some training images

In [ ]:
# get some random training images
dataiter = iter(train_loader)
# next(dataiter)

images, labels = dataiter.next()

# show images

# functions to show an image
def imshow(img):
    """
    :param img: (PyTorch Tensor)
    """
    # unnormalize
    img = img / 2 + 0.5     
    # Convert tensor to numpy array
    npimg = img.numpy()
    plt.figure(figsize = (20,8))
    # Color channel first -> color channel last
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

  
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('{:>10}'.format(classes[labels[j]]) for j in range(batch_size)))

In [ ]:
images[0].shape

In [ ]:
images.float().dtype
images.shape


Note that only ``train_data`` uses ``transform_train``, while
``val_data`` and ``test_data`` use ``transform_test`` to produce deterministic
results for evaluation.

## Train

### Initialize Optimizer and Some Helpers

In [ ]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    # it combines softmax with negative log likelihood loss
    criterion = nn.CrossEntropyLoss()  
    #optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    return criterion, optimizer

In [ ]:
# duplicate from before, but maybe it's useful to be able to change the batch size dynamically during training...? mini-batches?

def get_train_loader(batch_size):
    return torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, sampler=None,
                                              num_workers=num_workers)

def get_val_loader(batch_size):
# Use larger batch size for validation to speed up computation
    return torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True, sampler=None,
                                          num_workers=num_workers)

### Training Loop

Following is the main training loop. It is the same as the loop in
`CIFAR10 <dive_deep_cifar10.html>`__
and ImageNet.

In [ ]:
def train(net, batch_size, n_epochs, learning_rate):
    """
    Train a neural network and print statistics of the training
    
    :param net: (PyTorch Neural Network)
    :param batch_size: (int)
    :param n_epochs: (int)  Number of iterations on the training set
    :param learning_rate: (float) learning rate used by the optimizer
    """
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("n_epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    train_loader = get_train_loader(batch_size)
    n_minibatches = len(train_loader)
    print("number of minibaches: ", n_minibatches)

    criterion, optimizer = createLossAndOptimizer(net, learning_rate)
    # Init variables used for plotting the loss
    train_history = []
    val_history = []
    # train_history_acc = []
    # val_history_acc = []

    training_start_time = time.time()
    best_error = np.inf
    best_model_path = "best_model.pth"

    # mutli-gpu
    if gpus is not None and len(gpus) > 1:
        print('multi-gpu ', gpus)
        net = torch.nn.DataParallel(net, device_ids=gpus)  # model becomes `torch.nn.DataParallel` w/ model.module being the orignal `torch.nn.Module`
        net = net.to(device)

    # Move model to gpu if possible
    else:
        net = net.to(device)

    for epoch in range(n_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        print_every = n_minibatches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, (inputs, labels) in enumerate(train_loader):

            # Move tensors to correct device
            inputs, labels = inputs.to(device=device, dtype=torch.float), labels.to(device=device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            total_train_loss += loss.item()

            # print every 10th of epoch
            if (i + 1) % (print_every + 1) == 0:    
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                      epoch + 1, int(100 * (i + 1) / n_minibatches), running_loss / print_every,
                      time.time() - start_time))
                running_loss = 0.0
                start_time = time.time()

        train_history.append(total_train_loss / len(train_loader))

        total_val_loss = 0
        # Do a pass on the validation set
        # We don't need to compute gradient,
        # we save memory and computation using torch.no_grad()
        with torch.no_grad():
            val_loader = get_val_loader(batch_size)
            for inputs, labels in val_loader:
                # Move tensors to correct device
                inputs, labels = inputs.to(device=device, dtype=torch.float), labels.to(device=device)
                # Forward pass
                predictions = net(inputs)
                val_loss = criterion(predictions, labels)
                total_val_loss += val_loss.item()
            
        val_history.append(total_val_loss / len(val_loader))
        # Save model that performs best on validation set
        if total_val_loss < best_error:
            best_error = total_val_loss
            torch.save(net.state_dict(), best_model_path)

        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))

    print("Training Finished, took {:.2f}s".format(time.time() - training_start_time))
    
    # Load best model
    net.load_state_dict(torch.load(best_model_path))
    
    return train_history, val_history

### Initialize Model and Run

In [ ]:
# https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab/48919022
src = list(files.upload().values())[0]
open('convgru.py','wb').write(src)
# print(os.path.abspath('utils.py'))
import utils
# help(utils)

In [ ]:
from convgru import *

net = ConvGRU(
                #input_size=(height, width),
                in_channels=3,
                hidden_channels=[20, 50],
                kernel_size=(3, 3),
                num_layers=2,
                batch_first=True,
                bias=True,
                return_all_layers=False)

In [ ]:
train_history, val_history = train(net, batch_size=batch_size, n_epochs=10, learning_rate=0.001)

Evaluate the accuracy

In [ ]:
def dataset_accuracy(net, data_loader, name="", print_first_k_mistakes=0):
    net = net.to(device)
    correct = 0
    total = 0
    to_print = []
    for images_loader, labels_loader in data_loader:
        images, labels = images_loader.to(device=device, dtype=torch.float), labels_loader.to(device=device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        if print_first_k_mistakes > 0:
            for idx, match in enumerate(zip(predicted, labels)):
                if match[0] != match[1]:
                    error = "Actually unpolarized:"
                    if match[1] == 1:
                        error = "Actually polarized:"
                    to_print.append((error, images_loader[idx]))
                    print_first_k_mistakes -= 1
    accuracy = 100 * float(correct) / total
    print('Accuracy of the network on the {} {} images: {:.2f} %'.format(total, name, accuracy))

    if len(to_print) > 0:
        print('\n'.join('{:>10}'.format(label[0]) for label in to_print))
        imshow(torchvision.utils.make_grid([img[1] for img in to_print]))

def train_set_accuracy(net):
    dataset_accuracy(net, train_loader, "train")

def val_set_accuracy(net):
    dataset_accuracy(net, val_loader, "validation")  
    
def test_set_accuracy(net):
    dataset_accuracy(net, test_loader, "test", 10)

def compute_accuracy(net):
    train_set_accuracy(net)
    val_set_accuracy(net)
    test_set_accuracy(net)
    
print("Computing accuracy...")
compute_accuracy(net)

### Plot results (these need to be refactored)


In [ ]:
def plot_losses(train_history, val_history):
    x = np.arange(1, len(train_history) + 1)

    plt.figure(figsize=(8, 6), dpi=100)
    plt.plot(x, train_history, '-r', label="Training loss", linewidth=2)
    plt.plot(x, val_history, '-b', label="Validation loss", linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.title("Training and Validation Loss History" + "(" + z_agg_mode + ")")
    plt.show()

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    :param cm: (numpy matrix) confusion matrix
    :param classes: [str]
    :param normalize: (bool)
    :param title: (str)
    :param cmap: (matplotlib color map)
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 8), dpi=100)   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# plt.figure()
# plt.plot(np.arange(0,epochs,1),train_acc_lst,'r')
# plt.ylabel('Training Accuracy')
# plt.xlabel('Epoch')
# plt.title('CNN Classify Fluo Polarization')
# plt.show()

In [ ]:
# plt.figure()
# plt.plot(np.arange(0,epochs,1),train_loss_lst,'r')
# plt.ylabel('Training Loss')
# plt.xlabel('Epoch')
# plt.title('CNN Classify Fluo Polarization')
# plt.show()

plot_losses(train_history, val_history)

In [ ]:
# plt.figure()
# plt.plot(np.arange(0,epochs,1),val_acc_lst,'r')
# plt.ylabel('Validation Accuracy')
# plt.xlabel('Epoch')
# plt.title('CNN Classify Fluo Polarization')
# plt.show()

In [ ]:
import itertools

test_batch_size = batch_size

def accuracy_per_class(net):
    net = net.to(device)
    n_classes = 2
    # (real, predicted)
    confusion_matrix = np.zeros((n_classes, n_classes), dtype=np.int64)

    for images, labels in test_loader:
        images, labels = images.to(device=device, dtype=torch.float), labels.to(device=device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(test_batch_size):
            confusion_matrix[labels[i], predicted[i]] += 1
            label = labels[i]

    print("{:<10} {:^10}".format("Class", "Accuracy (%)"))
    for i in range(n_classes):
        class_total = confusion_matrix[i, :].sum()
        class_correct = confusion_matrix[i, i]
        percentage_correct = 100.0 * float(class_correct) / class_total
        
        print('{:<10} {:^10.2f}'.format(classes[i], percentage_correct))
    return confusion_matrix

confusion_matrix = accuracy_per_class(net)

In [ ]:
# Plot normalized confusion matrix
plot_confusion_matrix(confusion_matrix, classes, normalize=True,
                      title='Normalized confusion matrix')

# Plot non-normalized confusion matrix
plot_confusion_matrix(confusion_matrix, classes,
                      title='Confusion matrix, without normalization')